# Ensemble Techniques And Its Types-5

#### Q1. You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values.
* Design a pipeline that includes the following steps:
    * Use an automated feature selection method to identify the important features in the dataset.
    * Create a numerical pipeline that includes the following steps:
        * Impute the missing values in the numerical columns using the mean of the column values.
        * Scale the numerical columns using standardization.
    * Create a categorical pipeline that includes the following steps:
        * Impute the missing values in the categorical columns using the most frequent value of the column.
        * One-hot encode the categorical columns.
    * Combine the numerical and categorical pipelines using a ColumnTransformer.
    * Use a Random Forest Classifier to build the final model.
    * Evaluate the accuracy of the model on the test dataset.
* **Note** *Your solution should include code snippets for each step of the pipeline, and a brief explanation of each step. You should also provide an interpretation of the results and suggest possible improvements for the pipeline.*

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score

In [2]:
# Loading Titanic Dataset
url = 'https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv'
ds = pd.read_csv(url)
ds.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


In [3]:
# Checking various variables
print(ds.isnull().sum())
ds.info()

Survived                   0
Pclass                     0
Name                       0
Sex                        0
Age                        0
Siblings/Spouses Aboard    0
Parents/Children Aboard    0
Fare                       0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Survived                 887 non-null    int64  
 1   Pclass                   887 non-null    int64  
 2   Name                     887 non-null    object 
 3   Sex                      887 non-null    object 
 4   Age                      887 non-null    float64
 5   Siblings/Spouses Aboard  887 non-null    int64  
 6   Parents/Children Aboard  887 non-null    int64  
 7   Fare                     887 non-null    float64
dtypes: float64(2), int64(4), object(2)
memory usage: 55.6+ KB


In [4]:
# Prepartion Data for model
x = ds.drop(columns=['Survived','Name'])
y = ds['Survived']
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=47, stratify = y)

# Defining numerical and categorical columns
num_col = x.select_dtypes(include=['number']).columns.tolist()
cat_col = x.select_dtypes(exclude=['number']).columns.tolist()

# Feature Selection
fs = SelectFromModel(RandomForestClassifier(n_estimators = 100, random_state= 42))

# Numerical & Categorical Pipline
num=Pipeline(steps=[('imputer',SimpleImputer(strategy="mean")),('scaler',StandardScaler())])
cat=Pipeline(steps=[('imputer',SimpleImputer(strategy="most_frequent")),('onehot',OneHotEncoder())])

# Creating the ColumnTransformer without fitting it
com = ColumnTransformer(transformers=[('num', num, num_col), ('cat', cat, cat_col)])

# Creating the Pipeline
pl = Pipeline(steps=[("Feature_selector", fs), 
                     ("classifier", RandomForestClassifier(n_estimators=100, random_state=42))])

# Fit the preprocessing steps on the training data
X_train = com.fit_transform(x_train)
X_test = com.transform(x_test)

# Fitting and Evaluation
pl.fit(X_train,y_train)
y_pred = pl.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print(f"Accuracy: {acc:.4f}")

Accuracy: 0.7658


#### Q2. Build a pipeline that includes a Random Forest Classifier and a Logistic Regression Classifier, and then use a Voting Classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

# Loading Dataset & Spliting dataset into train test
ds = load_iris()
x = ds.data
y = ds.target
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.15, random_state=17)

# Creating models
lr=LogisticRegression(random_state=92)
rf=RandomForestClassifier(n_estimators=100, random_state=62)
com = VotingClassifier(estimators=[('rf', rf),('lr',lr)], voting='hard')

# Fitting and Evaluating the model
com.fit(x_train,y_train)
y_pred = com.predict(x_test)
acc = accuracy_score(y_test,y_pred)
print(f"Accuracy:{acc:.4f}")

Accuracy:0.9130


C:\Users\SALESKEN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
